<a href="https://colab.research.google.com/github/Mohd6288/DAMLCP_Project/blob/main/text2image_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)

import os
os.chdir('/content/drive/MyDrive/python') # to change to another directory

Mounted at /content/drive/


In [2]:
from pathlib import Path
from huggingface_hub import notebook_login
if not (Path.home()/'.huggingface'/'token').exists():
    notebook_login()

In [3]:
import sys

if 'google.colab' in sys.modules:
    !pip install --upgrade transformers diffusers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 103.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [6]:
!pip install gradio
# !pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import requests
from io import BytesIO
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

import torch

# Get cpu, gpu or mps device for training.
# See: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#creating-models
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import LDMSuperResolutionPipeline
import gradio as gr

The convenience function to display a batch of images.

In [3]:
# https://matplotlib.org/stable/gallery/axes_grid1/simple_axesgrid.html
from mpl_toolkits.axes_grid1 import ImageGrid

def plot_images(imgs, rows=1, cols=None, figsize=(12,8), title=None):
    fig = plt.figure(figsize=figsize)           # control figure size
    grid = ImageGrid(
        fig, 111,                                                     # similar to subplot(111) | see: https://stackoverflow.com/a/11404223
        nrows_ncols=(rows, cols if cols is not None else len(imgs)),  # control rows/cols
        axes_pad=0.1,                                                 # pad between axes in inch
    )
    if title is not None:               # https://matplotlib.org/3.2.1/gallery/subplots_axes_and_figures/figure_title.html
        fig.suptitle(title, x=0, y=0.5)

    for ax, im in zip(grid, imgs):      # Iterating over the grid returns the Axes.
        ax.set_xticks([])               # no x/y ticks: https://stackoverflow.com/a/45149018
        ax.set_yticks([])               #               https://stackoverflow.com/a/58535290
        ax.imshow(im)

In [4]:
MODEL_ID = "CompVis/stable-diffusion-v1-4" # same comment as in the first notebook

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_ID,
    revision="fp16",
    torch_dtype=torch.float16,
    safety_checker = None # remove NSFW filter
).to("cuda")

# Note: removing the filter is no licence to do harm, it is to give *you* the responsibility
# of your use. (Also, the HF safety_checker is very, very conservative, and rejects
# a lot of abstract images.)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:221: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

(…)kpoints/scheduler_config-checkpoint.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis-

In [ ]:
SAVE_MEMORY = False

if SAVE_MEMORY:                     # Saves memory at the cost of speed:
    pipe.enable_attention_slicing() #  https://huggingface.co/docs/diffusers/main/en/api/diffusion_pipeline#diffusers.DiffusionPipeline.enable_attention_slicing

if device=="mps":                           # First-time "warmup" pass for M1/M2 macs
    _ = pipe(prompt, num_inference_steps=1) # https://huggingface.co/docs/diffusers/v0.4.1/en/optimization/mps

## Experiments

- Can you build a pipeline that repeats the process of generating an image and then selecting one as a basis for the next?
- This is of course also valid for super-resolution, where it might be possible to get interesting results by applying hyperresolution serveral times in a row (perhaps upscale, then select a portion of the result image, upscale again...)..
- How do you display your results? Note that, as with GANs, if you create a series of images, you can then merge them into a video..
- These are fully compatible with Gradio (you can see the code for many apps using these models on [Huggingface Spaces](https://huggingface.co/spaces?sort=modified&search=diffusion), it could be cool to build your own, or modify an existing one!

In [5]:
# Set up the model
MODEL_ID = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_ID,
    revision="fp16",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

# Pre-load an initial image
init_image_url = "https://i.pinimg.com/736x/c3/09/4c/c3094c5fbb257f0e794a838247f7ef5f.jpg"
init_image = Image.open(requests.get(init_image_url, stream=True).raw).convert("RGB")

# Function to run the pipeline
def generate_images(prompt, strength, num_images, num_steps):
    """
    Generate images using the Stable Diffusion pipeline.
    Args:
        prompt (str): Text prompt for the image generation.
        strength (float): Strength parameter (0 to 1).
        num_images (int): Number of images to generate.
        num_steps (int): Number of inference steps.
    Returns:
        List of generated PIL images.
    """
    torch.manual_seed(1000)  # Set manual seed for reproducibility
    results = pipe(
        prompt=prompt,
        num_images_per_prompt=num_images,
        image=[init_image] * num_images,
        strength=strength,
        num_inference_steps=num_steps
    )
    return results.images

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline

In [ ]:
# Function to run the pipeline and save GIF
def generate_images_with_gif(prompt, strength, num_images, num_steps):
    """
    Generate images using the Stable Diffusion pipeline and create a GIF.
    Args:
        prompt (str): Text prompt for the image generation.
        strength (float): Strength parameter (0 to 1).
        num_images (int): Number of images to generate.
        num_steps (int): Number of inference steps.
    Returns:
        List of file paths to generated images and GIF file path.
    """
    torch.manual_seed(1000)  # Set manual seed for reproducibility
    results = pipe(
        prompt=prompt,
        num_images_per_prompt=num_images,
        image=[init_image] * num_images,
        strength=strength,
        num_inference_steps=num_steps
    )

    # Save images to disk
    image_paths = []
    images = results.images
    for i, img in enumerate(images):
        file_path = f"generated_image_{i}.png"
        img.save(file_path)
        image_paths.append(file_path)

    # Create a GIF
    gif_path = "generated_images.gif"
    images[0].save(
        gif_path, save_all=True, append_images=images[1:], duration=500, loop=0
    )

    return image_paths, gif_path

# Define the Gradio interface
def gradio_pipeline_with_gif(prompt, strength, num_images, num_steps):
    image_paths, gif_path = generate_images_with_gif(prompt, strength, num_images, num_steps)
    return image_paths, gif_path

interface = gr.Interface(
    fn=gradio_pipeline_with_gif,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your prompt here", label="Prompt"),
        gr.Slider(0.0, 1.0, value=0.8, step=0.01, label="Strength"),
        gr.Slider(1, 5, value=3, step=1, label="Number of Images"),
        gr.Slider(10, 100, value=50, step=1, label="Inference Steps")
    ],
    outputs=[
        gr.Gallery(label="Generated Images"),
        gr.File(label="Download Generated GIF")
    ]
)

interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://97403f5cd3bc17ad83.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/40 [00:00<?, ?it/s]

Created dataset file at: .gradio/flagged/dataset1.csv
